In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [17]:
import re
import numpy as np
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [18]:
class_matrix = []

main_class = [100, 200, 300, 400]
sub_class = [101, 102, 103, 104, 105, 109, 110, 115, 119, 120, 121, 122, 123, 127, 128, 130, 131, 133, 199, 235, 236, 238, 241, 245, 248, 252, 254, 258, 259, 299, 361, 399, 401, 494, 495, 496, 499]

with open('IE506_2024_progchallenge_train.txt', 'r') as file:

    for i,line in enumerate(file):
        line_list = line.split(" ")

        M = line_list[0]
        S = line_list[1]

        # Extract numerical values using regular expressions
        M = re.findall(r'\d+', M)
        S = re.findall(r'\d+', S)

        # Convert the extracted numbers to integers
        M = [int(num) for num in M]
        S  = [int(num) for num in S]

        main_class_vec = [1 if label in M else 0 for label in main_class]
        sub_class_vec = [1 if label in S else 0 for label in sub_class]

        # Concatenate main class and subclass vectors
        class_vector = main_class_vec + sub_class_vec
        
        # Append the class vector to the class matrix
        class_matrix.append(class_vector)

y_matrix = np.array(class_matrix)

In [19]:
y_matrix.shape

(200000, 41)

In [20]:
# Initialize an empty list to store the feature-value pairs
data = []
row_indices = []
col_indices = []

# Read the data file and populate the data list
with open('IE506_2024_progchallenge_train.txt', 'r') as file:
    for i, line in enumerate(file):
        line_parts = line.split(" ")
        for pair in line_parts[2:]:
            feature_id, feature_value = pair.split(':')
            row_indices.append(i)
            col_indices.append(int(feature_id))
            data.append(float(feature_value))

# Create a sparse matrix from the collected data
sparse_matrix = csr_matrix((data, (row_indices, col_indices)), shape=(200000, 47236))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sparse_matrix, y_matrix, test_size=0.2, random_state=42)

In [21]:
X_train.shape

(160000, 47236)

In [8]:
params = {
    'max_depth': 6,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
    'n_estimators': 1000,
    'gamma': 0,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
}


In [10]:
# Initialize XGBoost classifier with specified parameters
xgb_classifier = XGBClassifier(**params)

# Train the classifier
xgb_classifier.fit(X_train, y_train)

In [ ]:
import pickle

# Save the model to a file
with open('xgb_classifier_model2.pkl', 'wb') as file:
    pickle.dump(xgb_classifier, file)

print("Model saved successfully.")

In [22]:
import pickle

# Load the model from the file
with open('xgb_classifier_model2.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

print("Model loaded successfully.")

Model loaded successfully.


In [26]:
from scipy.sparse import dok_matrix
 
# Initialize a dictionary of keys (DOK) matrix
X_test_final = dok_matrix((150000, 47236), dtype=np.float32)
 
with open('IE506_2024_progchallenge_test.txt', 'r') as file:
    for i, line in enumerate(file):
        line_list = line.split(" ")
        for s in line_list:
            id_, val = s.split(':')
            id_ = int(id_)
            val = float(val)
            X_test_final[i, id_] = val

X_test = X_test_final.tocsr()

In [27]:
X_test.shape

(150000, 47236)

In [7]:
import pickle

# Load the model from the file
with open('xgb_classifier_model2.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

print("Model loaded successfully.")

Model loaded successfully.


In [29]:
# Make predictions
y_pred = loaded_model.predict(X_test)

In [30]:
y_pred.shape

(150000, 41)

In [31]:
cls = [100, 200, 300, 400, 101, 102, 103, 104, 105, 109, 110, 115, 119, 120, 121, 122, 123, 127, 128, 130, 131, 133, 199, 235, 236, 238, 241, 245, 248, 252, 254, 258, 259, 299, 361, 399, 401, 494, 495, 496, 499]

In [32]:
ID = []
M = []
S = []

for i in range(150000):
    ID.append(i)
    temp_M = []
    temp_S = []
    for j in range(41):
        if 0 <= j <= 3:
            if y_pred[i,j] == 1:
                temp_M.append(str(cls[j]))

        elif 4 <= j :
            if y_pred[i,j] == 1:
                temp_S.append(str(cls[j]))

    M.append(temp_M)
    S.append(temp_S)

In [33]:
df = pd.DataFrame({'ID' : ID,
                    'M' : M,
                    'S' : S})

In [34]:
df.head()

,ID,M,S
0,0,[100],[105]
1,1,[100],[105]
2,2,"[100, 300]",[399]
3,3,[400],"[495, 496]"
4,4,[],[]


In [35]:
import os

directory = "submission"
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the DataFrame as a CSV file
filename = os.path.join(directory, f"sub56.csv")
df.to_csv(filename, index=False)
print('Saved csv file')


Saved csv file
